This notebook is just to use `timeit` to time small functions and figure out what are the slowest in the package. It is not for demonstrative purposes.

In [ ]:
import frispy
import timeit

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

In [ ]:
def do_timeit(fname, number=100):
    setup = f"from __main__ import {fname}"
    t = timeit.timeit(f"{fname}()", number=number, setup=setup)
    return f"{fname}: {number} iterations took {t:.4f} seconds; {t/number:.2e} s/iter"

In [ ]:
trajectory = frispy.Trajectory()
phi = np.pi / 2.
theta = np.pi / 4.
vel = np.random.rand(3)
w = np.random.rand(3)

def calc_intermediate_quantities():
    _ = trajectory.calculate_intermediate_quantities(phi, theta, vel, w)

In [ ]:
def calc_R():
    _ = trajectory.rotation_matrix(0.12324, 0.9874)

print(do_timeit("calc_R", 626))

In [ ]:
print(do_timeit("calc_intermediate_quantities", 626))

In [ ]:
eom = frispy.EOM()
res = eom.compute_forces(phi, theta, vel, w)

def compute_forces():
    _ = eom.compute_forces(phi, theta, vel, w)

def compute_torques():
    _ = eom.compute_torques(vel, w, res)

In [ ]:
print(do_timeit("compute_forces", 626))

In [ ]:
print(do_timeit("compute_torques", 626))

In [ ]:
d = frispy.Disc()
coords = d.initial_conditions_as_ordered_list

def compute_derivatives():
    _ = eom.compute_derivatives(0, np.array(coords))

def compute_trajectory():
    _ = d.compute_trajectory()

In [ ]:
print(do_timeit("compute_derivatives", 626))

In [ ]:
print(do_timeit("compute_trajectory", 10))

In [ ]:
from scipy.integrate import solve_ivp, odeint

def call_solver():
    _ = solve_ivp(
        fun=d.eom.compute_derivatives,
        t_span=(0, 3),
        y0=d.initial_conditions_as_ordered_list,
    )

In [ ]:
print(do_timeit("call_solver", 10))

In [ ]:
y0 = d.initial_conditions_as_ordered_list
# y0[3] = 13.
# y0[0] = 0.
print(y0)

In [ ]:
r = solve_ivp(
    fun=d.eom.compute_derivatives,
    t_span=(0, 3),
    y0=y0,
)

In [ ]:
y0[0] = 0
y0[3] = 13.
def call_solver2():
    _ = solve_ivp(
        fun=d.eom.compute_derivatives,
        t_span=(0, 3),
        y0=y0,
#        first_step=0.05,
        rtol=2e-2,
        atol=5e-3,
#        method="LSODA"
    )

In [ ]:
print(do_timeit("call_solver2", 100))

In [ ]:
t = np.linspace(0, 3, 1000)

def compute_derivs_static(coords, time, disc):
    return disc.eom.compute_derivatives(time, coords)

def call_odeint():
    _ = odeint(compute_derivs_static, t=t, y0=y0, args=(d,))

In [ ]:
print(do_timeit("call_odeint", 10))